In [ ]:
%%writefile kmeans_cpu_serial.c
/*
  kmeans_cpu_serial.c
  Serial (single-thread) K-means with per-iteration profiling suitable for comparison
  with the CUDA version. Outputs:
    - labels_cpu_serial.csv
    - centroids_cpu_serial.csv
    - profile_cpu_serial.csv
    - meta_cpu_serial.txt

  Build:
    gcc -O3 -march=native kmeans_cpu_serial.c -o kmeans_cpu_serial -lm

  Run:
    ./kmeans_cpu_serial N D K max_iter tol init_method verbose profile_csv

  Defaults:
    N=20000, D=16, K=10, max_iter=200, tol=1e-4, init_method=1 (kmeans++), verbose=1
*/

#define _POSIX_C_SOURCE 200809L
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include <time.h>
#include <unistd.h>

#ifndef M_PI
#define M_PI 3.14159265358979323846
#endif

static double now_ms() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec*1000.0 + ts.tv_nsec/1.0e6;
}

/* xorshift64* RNG (same as other binaries) */
static unsigned long long rng_state = 88172645463325252ULL;
static inline unsigned long long xorshift64star() {
    unsigned long long x = rng_state;
    x ^= x >> 12; x ^= x << 25; x ^= x >> 27;
    rng_state = x;
    return x * 2685821657736338717ULL;
}
static inline double drand01() {
    return (xorshift64star() >> 11) * (1.0/9007199254740992.0);
}

/* Synthetic data generator (same logic as CUDA and OpenMP versions) */
static void generate_synthetic(double *data, int N, int D, int K) {
    double *centers = (double*)malloc((size_t)K*D*sizeof(double));
    for (int k=0;k<K;++k)
        for (int d=0; d<D; ++d)
            centers[k*D + d] = (drand01()*2.0 - 1.0) * 10.0;

    for (int i=0;i<N;++i) {
        srand(123);
        int c = rand() % K;
        for (int d=0; d<D; ++d) {
            double u1 = drand01(), u2 = drand01();
            double r = sqrt(-2.0 * log(fmax(u1,1e-12)));
            double z = r * cos(2*M_PI*u2);
            double scale = 1.0 + 0.5 * drand01();
            data[i*D + d] = centers[c*D + d] + z * scale;
        }
    }
    free(centers);
}

/* squared distance */
static inline double sqdist(const double *p, const double *c, int D) {
    double s = 0.0;
    for (int d=0; d<D; ++d) {
        double diff = p[d] - c[d];
        s += diff*diff;
    }
    return s;
}

/* kmeans++ init (same as GPU host path) */
static void init_kmeanspp(const double *data, double *centroids, int N, int D, int K) {
    int first = (int)(drand01() * N);
    for (int d=0; d<D; ++d) centroids[0*D + d] = data[first*D + d];

    double *dist2 = (double*)malloc((size_t)N*sizeof(double));
    for (int i=0;i<N;++i) dist2[i] = sqdist(&data[i*D], &centroids[0*D], D);

    for (int k=1;k<K;++k) {
        double sum = 0.0;
        for (int i=0;i<N;++i) sum += dist2[i];
        double r = drand01() * sum;
        double csum = 0.0;
        int chosen = N-1;
        for (int i=0;i<N;++i) {
            csum += dist2[i];
            if (csum >= r) { chosen = i; break; }
        }
        for (int d=0; d<D; ++d) centroids[k*D + d] = data[chosen*D + d];

        for (int i=0;i<N;++i) {
            double v = sqdist(&data[i*D], &centroids[k*D], D);
            if (v < dist2[i]) dist2[i] = v;
        }
    }
    free(dist2);
}

/* Save helpers (distinct filenames to avoid clashes) */
static void save_labels(const char *path, const int *labels, int N) {
    FILE *f = fopen(path, "w"); if (!f) { perror(path); return; }
    for (int i=0;i<N;++i) fprintf(f, "%d\n", labels[i]);
    fclose(f);
}
static void save_centroids(const char *path, const double *centroids, int K, int D) {
    FILE *f = fopen(path, "w"); if (!f) { perror(path); return; }
    for (int k=0;k<K;++k) {
        for (int d=0; d<D; ++d) {
            fprintf(f, "%.8f", centroids[k*D + d]);
            if (d < D-1) fputc(',', f);
        }
        fputc('\n', f);
    }
    fclose(f);
}

/* Serial K-means with per-iteration profiling and CSV output */
static void kmeans_run_serial(
    const double *data, double *centroids, int *labels,
    int N, int D, int K, int max_iter, double tol, int verbose, const char *profile_csv,
    double *out_inertia, int *out_iters)
{
    FILE *pf = NULL;
    if (profile_csv && strlen(profile_csv) > 0) {
        pf = fopen(profile_csv, "w");
        if (pf) fprintf(pf, "iter,t_assign_ms,t_update_ms,t_inertia_ms,t_iter_ms\n");
    }

    double inertia = 0.0;
    int iters = 0;
    for (int iter=0; iter<max_iter; ++iter) {
        double t_iter0 = now_ms();

        /* assignment + accumulation (serial) */
        double t0 = now_ms();
        // zero accumulators
        double *sums = (double*)calloc((size_t)K*D, sizeof(double));
        int *counts = (int*)calloc((size_t)K, sizeof(int));

        for (int i=0;i<N;++i) {
            const double *p = &data[i*D];
            int best = 0;
            double bestd = sqdist(p, &centroids[0*D], D);
            for (int k=1;k<K;++k) {
                double d = sqdist(p, &centroids[k*D], D);
                if (d < bestd) { bestd = d; best = k; }
            }
            labels[i] = best;
            counts[best] += 1;
            for (int d=0; d<D; ++d) sums[best*D + d] += p[d];
        }
        double t_assign = now_ms() - t0;

        /* update centroids */
        t0 = now_ms();
        double movement = 0.0;
        for (int k=0;k<K;++k) {
            if (counts[k] > 0) {
                for (int d=0; d<D; ++d) {
                    double newc = sums[k*D + d] / (double)counts[k];
                    double diff = centroids[k*D + d] - newc;
                    movement += diff*diff;
                    centroids[k*D + d] = newc;
                }
            } else {
                /* keep old if empty */
            }
        }
        double t_update = now_ms() - t0;

        /* inertia */
        t0 = now_ms();
        inertia = 0.0;
        for (int i=0;i<N;++i) {
            int lbl = labels[i];
            inertia += sqdist(&data[i*D], &centroids[lbl*D], D);
        }
        double t_inertia = now_ms() - t0;

        double t_iter = now_ms() - t_iter0;
        if (pf) fprintf(pf, "%d,%.6f,%.6f,%.6f,%.6f\n", iter+1, t_assign, t_update, t_inertia, t_iter);

        if (verbose) {
            printf("Iter %3d | assign=%.3f ms update=%.3f ms inertia=%.3f ms | total=%.3f ms | move=%.6g\n",
                   iter+1, t_assign, t_update, t_inertia, t_iter, movement);
        }

        free(sums); free(counts);

        iters = iter + 1;
        if (movement < tol) {
            if (verbose) printf("Converged (movement %.6g <= tol %.6g)\n", movement, tol);
            break;
        }
    }

    if (pf) fclose(pf);
    *out_inertia = inertia;
    *out_iters = iters;
}

int main(int argc, char** argv) {
    int N = 20000, D = 16, K = 10, max_iter = 200, verbose = 1;
    double tol = 1e-4;
    int init_method = 1;
    const char *profile_csv = "profile_cpu_serial.csv";

    if (argc >= 2) N = atoi(argv[1]);
    if (argc >= 3) D = atoi(argv[2]);
    if (argc >= 4) K = atoi(argv[3]);
    if (argc >= 5) max_iter = atoi(argv[4]);
    if (argc >= 6) tol = atof(argv[5]);
    if (argc >= 7) init_method = atoi(argv[6]);
    if (argc >= 8) verbose = atoi(argv[7]);
    if (argc >= 9) profile_csv = argv[8];

    rng_state = 88172645463325252ULL;

    printf("Serial K-means CPU: N=%d D=%d K=%d max_iter=%d tol=%g init=%s\n",
           N, D, K, max_iter, tol, init_method ? "kmeans++" : "random");

    long nprocs = (long)sysconf(_SC_NPROCESSORS_ONLN);
    printf("Host reports CPU cores available: %ld\n", nprocs);

    /* allocate host data */
    double *h_data = (double*)malloc((size_t)N*D*sizeof(double));
    double *h_centroids = (double*)malloc((size_t)K*D*sizeof(double));
    int *h_labels = (int*)malloc((size_t)N*sizeof(int));
    if (!h_data || !h_centroids || !h_labels) { fprintf(stderr, "alloc failed\n"); return 1; }

    printf("Generating synthetic data on host...\n");
    generate_synthetic(h_data, N, D, K);

    if (init_method) {
        printf("Initializing centroids with kmeans++\n");
        init_kmeanspp(h_data, h_centroids, N, D, K);
    } else {
        // random init simple fallback
        for (int k=0;k<K;++k) {
            int idx = (int)(drand01() * N);
            for (int d=0; d<D; ++d) h_centroids[k*D + d] = h_data[idx*D + d];
        }
    }

    double t0 = now_ms();
    double inertia = 0.0; int iters = 0;
    kmeans_run_serial(h_data, h_centroids, h_labels, N, D, K, max_iter, tol, verbose, profile_csv, &inertia, &iters);
    double t1 = now_ms();

    printf("Serial K-means finished: iters=%d inertia=%.6g total_time=%.3f ms\n", iters, inertia, t1-t0);

    /* save outputs with distinct names to avoid clashes */
    save_labels("labels_cpu_serial.csv", h_labels, N);
    save_centroids("centroids_cpu_serial.csv", h_centroids, K, D);

    /* write meta file */
    FILE *mf = fopen("meta_cpu_serial.txt", "w");
    if (mf) { fprintf(mf, "N %d\nD %d\nK %d\n", N, D, K); fclose(mf); }

    printf("Saved labels_cpu_serial.csv, centroids_cpu_serial.csv, profile_cpu_serial.csv, meta_cpu_serial.txt\n");

    free(h_data); free(h_centroids); free(h_labels);
    return 0;
}

Writing kmeans_cpu_serial.c


In [ ]:
!gcc -O3 -march=native kmeans_cpu_serial.c -o kmeans_cpu_serial -lm
!ls -lh kmeans_cpu_serial

-rwxr-xr-x 1 root root 21K Nov 29 22:08 kmeans_cpu_serial


In [ ]:
import subprocess
import re
import csv
import time

# Ns to test for CPU serial
Ns = [100, 500, 1000, 5000, 10000, 20000, 50000, 100000, 200000, 300000, 500000, 1000000, 2000000]   # add more if you want

D = 16
K = 10
max_iter = 200
tol = "1e-4"
init_method = 1    # kmeans++
verbose = 0        # TURN THIS OFF FOR SPEED

results = []

print("Running CPU Serial K-means benchmarks...\n")

for N in Ns:
    print(f"▶ Running N = {N:,} ...")

    cmd = ["./kmeans_cpu_serial",
           str(N), str(D), str(K),
           str(max_iter), tol,
           str(init_method), str(verbose),
           f"profile_cpu_serial_{N}.csv"]

    # run & capture text output
    start = time.time()
    out = subprocess.check_output(cmd, universal_newlines=True)
    end = time.time()

    # Extract "total_time=xxx ms"
    match = re.search(r"total_time=([\d.]+)\s*ms", out)
    total_ms = float(match.group(1)) if match else None

    results.append((N, total_ms))
    print(f"   ✔ Done: {total_ms} ms\n")

print("🎉 All CPU serial runs complete!\n")

# Save summary to CSV
with open("runtime_cpu_serial.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["N", "time_ms"])
    writer.writerows(results)

print("Saved results to runtime_cpu_serial.csv\n")

# Print summary
print("===== SUMMARY (CPU SERIAL) =====")
for N, t in results:
    print(f"N={N:>10} → {t:.3f} ms")


Running CPU Serial K-means benchmarks...

▶ Running N = 100 ...
   ✔ Done: 0.296 ms

▶ Running N = 500 ...
   ✔ Done: 1.841 ms

▶ Running N = 1,000 ...
   ✔ Done: 3.171 ms

▶ Running N = 5,000 ...
   ✔ Done: 106.82 ms

▶ Running N = 10,000 ...
   ✔ Done: 180.07 ms

▶ Running N = 20,000 ...
   ✔ Done: 379.131 ms

▶ Running N = 50,000 ...
   ✔ Done: 1167.851 ms

▶ Running N = 100,000 ...
   ✔ Done: 3600.485 ms

▶ Running N = 200,000 ...
   ✔ Done: 5760.269 ms

▶ Running N = 300,000 ...
   ✔ Done: 10413.472 ms

▶ Running N = 500,000 ...
   ✔ Done: 15306.132 ms

▶ Running N = 1,000,000 ...


KeyboardInterrupt: 

In [ ]:
%%writefile kmeans_cuda_prof.cu
// kmeans_cuda_prof.cu
// GPU K-means with CUDA C (uses <cuda.h>), per-iteration timing, and CSV profiling.
// Outputs: labels_cuda.csv, centroids_cuda.csv, profile_cuda.csv, meta_cuda.txt
// Build:   nvcc -O3 -arch=sm_75 kmeans_cuda_prof.cu -o kmeans_cuda_prof
// Run:     ./kmeans_cuda_prof 20000 16 10 200 1e-4 256 1 profile_cuda.csv

#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cstring>
#include <string>
#include <cuda.h>
#include <cuda_runtime.h>

#ifndef M_PI
#define M_PI 3.14159265358979323846
#endif

// ----------------- Error check -----------------
#define CUDA_CHECK(e) do{ \
  cudaError_t err=(e); \
  if(err!=cudaSuccess){ \
    fprintf(stderr,"CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
    exit(1);} \
}while(0)

// ----------------- Simple timer -----------------
static double now_ms() {
  struct timespec ts;
  clock_gettime(CLOCK_MONOTONIC, &ts);
  return ts.tv_sec*1000.0 + ts.tv_nsec/1.0e6;
}

// ----------------- RNG (xorshift64*) -----------------
static unsigned long long rng_state = 88172645463325252ULL;
static inline unsigned long long xorshift64star() {
  unsigned long long x = rng_state;
  x ^= x >> 12; x ^= x << 25; x ^= x >> 27;
  rng_state = x;
  return x * 2685821657736338717ULL;
}
static inline double drand01() {
  return (xorshift64star() >> 11) * (1.0/9007199254740992.0);
}

// ----------------- Data generation (host) -----------------
static void generate_synthetic(double *data, int N, int D, int K) {
  double *centers = (double*)malloc((size_t)K*D*sizeof(double));
  for(int k=0;k<K;++k) for(int d=0;d<D;++d)
    centers[k*D+d] = (drand01()*2.0 - 1.0) * 10.0;

  for(int i=0;i<N;++i){
    srand(123);
    int c = rand() % K;
    for(int d=0; d<D; ++d){
      double u1=drand01(), u2=drand01();
      double r = sqrt(-2.0 * log(fmax(u1,1e-12)));
      double z = r * cos(2*M_PI*u2);
      double scale = 1.0 + 0.5*drand01();
      data[i*D + d] = centers[c*D + d] + z*scale;
    }
  }
  free(centers);
}

// ----------------- KMeans++ init (host) -----------------
static inline double sqdist_host(const double* p, const double* c, int D){
  double s=0.0; for(int d=0; d<D; ++d){ double df=p[d]-c[d]; s+=df*df; } return s;
}

static void init_kmeanspp(const double *data, double *centroids, int N, int D, int K) {
  int first = (int)(drand01() * N);
  memcpy(&centroids[0], &data[first*D], (size_t)D*sizeof(double));
  double *dist2 = (double*)malloc((size_t)N*sizeof(double));
  for(int i=0;i<N;++i) dist2[i] = sqdist_host(&data[i*D], &centroids[0], D);

  for(int k=1;k<K;++k){
    double sum=0.0; for(int i=0;i<N;++i) sum += dist2[i];
    double r=drand01()*sum, csum=0.0; int chosen=N-1;
    for(int i=0;i<N;++i){ csum += dist2[i]; if(csum>=r){chosen=i; break;} }
    memcpy(&centroids[k*D], &data[chosen*D], (size_t)D*sizeof(double));
    for(int i=0;i<N;++i){
      double v = sqdist_host(&data[i*D], &centroids[k*D], D);
      if(v<dist2[i]) dist2[i]=v;
    }
  }
  free(dist2);
}

// ----------------- Device info -----------------
static void print_device_info() {
  int dev=0; CUDA_CHECK(cudaGetDevice(&dev));
  cudaDeviceProp p; CUDA_CHECK(cudaGetDeviceProperties(&p, dev));
  printf("Device %d: %s\n", dev, p.name);
  printf("  CC %d.%d  SMs=%d  warpSize=%d  maxThreadsPerBlock=%d  sharedMemPerBlock=%zu KB\n",
         p.major, p.minor, p.multiProcessorCount, p.warpSize, p.maxThreadsPerBlock,
         p.sharedMemPerBlock/1024);
  printf("  memory: %.1f GB global, %.1f MB const\n",
         p.totalGlobalMem/1e9, p.totalConstMem/1e6);
}

// ----------------- Kernels -----------------
// assign + accumulate sums and counts + record dist2
// data: N x D (row-major), centroids: K x D
// sums: K x D (double), counts: K (int), labels: N
// dist2_out: per-point squared distance to nearest centroid
__global__ void assign_accumulate_kernel(
  const double* __restrict__ data,
  const double* __restrict__ centroids,
  int N, int D, int K,
  int* __restrict__ labels,
  double* __restrict__ sums,
  int* __restrict__ counts,
  double* __restrict__ dist2_out,
  int* __restrict__ thread_map_sample // size >= sampleN*3, stores [i,blockIdx.x,threadIdx.x] etc.
){
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i >= N) return;

  // record a small sample of (point_i, block, thread)
  if (thread_map_sample && i < 16) {
    int base = i*3;
    thread_map_sample[base+0] = i;
    thread_map_sample[base+1] = blockIdx.x;
    thread_map_sample[base+2] = threadIdx.x;
  }

  // find nearest centroid
  int best_k = 0;
  double bestd = 0.0;
  // distance to centroid 0
  {
    double s=0.0;
    const double* p = &data[i*D];
    const double* c = &centroids[0];
    for(int d=0; d<D; ++d){ double df=p[d]-c[d]; s+=df*df; }
    bestd = s;
  }
  for(int k=1;k<K;++k){
    double s=0.0;
    const double* p = &data[i*D];
    const double* c = &centroids[k*D];
    for(int d=0; d<D; ++d){ double df=p[d]-c[d]; s+=df*df; }
    if (s < bestd){ bestd=s; best_k=k; }
  }
  labels[i] = best_k;
  dist2_out[i] = bestd;

  // accumulate sums & counts (atomics on double require CC>=6.0)
  atomicAdd(&counts[best_k], 1);
  const double* p = &data[i*D];
  for(int d=0; d<D; ++d){
    atomicAdd(&sums[best_k*D + d], p[d]);
  }
}

// ----------------- Save helpers -----------------
static void save_labels(const char* path, const int* labels, int N) {
  FILE* f=fopen(path,"w"); if(!f){perror("labels_cuda.csv");return;}
  for(int i=0;i<N;++i) fprintf(f,"%d\n", labels[i]);
  fclose(f);
}
static void save_centroids(const char* path, const double* C, int K, int D) {
  FILE* f=fopen(path,"w"); if(!f){perror("centroids_cuda.csv");return;}
  for(int k=0;k<K;++k){
    for(int d=0; d<D; ++d){
      fprintf(f,"%.8f", C[k*D+d]);
      if(d<D-1) fputc(',', f);
    }
    fputc('\n', f);
  }
  fclose(f);
}

// ----------------- Main -----------------
int main(int argc, char** argv){
  // Args: N D K max_iter tol block_size verbose profile_csv
  int    N = 20000;
  int    D = 16;
  int    K = 10;
  int    max_iter = 200;
  double tol = 1e-4;
  int    block_size = 256;
  int    verbose = 1;
  const char* profile_csv = "profile_cuda.csv";

  if(argc>=2)  N = atoi(argv[1]);
  if(argc>=3)  D = atoi(argv[2]);
  if(argc>=4)  K = atoi(argv[3]);
  if(argc>=5)  max_iter = atoi(argv[4]);
  if(argc>=6)  tol = atof(argv[5]);
  if(argc>=7)  block_size = atoi(argv[6]);
  if(argc>=8)  verbose = atoi(argv[7]);
  if(argc>=9)  profile_csv = argv[8];

  rng_state = 88172645463325252ULL;

  print_device_info();
  printf("K-means CUDA: N=%d D=%d K=%d max_iter=%d tol=%g block=%d verbose=%d\n",
         N,D,K,max_iter,tol,block_size,verbose);

  // Host alloc
  size_t szX = (size_t)N*D*sizeof(double);
  size_t szC = (size_t)K*D*sizeof(double);
  size_t szL = (size_t)N*sizeof(int);
  double *h_data=(double*)malloc(szX);
  double *h_centroids=(double*)malloc(szC);
  int    *h_labels=(int*)malloc(szL);
  if(!h_data||!h_centroids||!h_labels){ fprintf(stderr,"Host alloc failed\n"); return 1; }

  // Generate & init
  printf("Generating synthetic data on host...\n");
  generate_synthetic(h_data, N, D, K);
  printf("Init (kmeans++) on host...\n");
  init_kmeanspp(h_data, h_centroids, N, D, K);

  // Device alloc
  double *d_data=nullptr, *d_centroids=nullptr, *d_sums=nullptr, *d_dist2=nullptr;
  int *d_counts=nullptr, *d_labels=nullptr, *d_threadmap=nullptr;
  CUDA_CHECK(cudaMalloc(&d_data, szX));
  CUDA_CHECK(cudaMalloc(&d_centroids, szC));
  CUDA_CHECK(cudaMalloc(&d_labels, szL));
  CUDA_CHECK(cudaMalloc(&d_sums, szC));
  CUDA_CHECK(cudaMalloc(&d_counts, K*sizeof(int)));
  CUDA_CHECK(cudaMalloc(&d_dist2, N*sizeof(double)));
  CUDA_CHECK(cudaMalloc(&d_threadmap, 16*3*sizeof(int))); // sample for first 16 points

  // Copy data to device
  CUDA_CHECK(cudaMemcpy(d_data, h_data, szX, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(d_centroids, h_centroids, szC, cudaMemcpyHostToDevice));

  // grid config
  int grid = (N + block_size - 1) / block_size;
  printf("Launch config: grid=%d blocks, block=%d threads  (total threads %d)\n",
         grid, block_size, grid*block_size);

  // CSV profile
  FILE* pf = fopen(profile_csv, "w");
  if(pf){ fprintf(pf, "iter,t_kernel_ms,t_update_ms,t_inertia_ms,t_iter_ms\n"); }

  // events for kernel timing
  cudaEvent_t evA, evB; CUDA_CHECK(cudaEventCreate(&evA)); CUDA_CHECK(cudaEventCreate(&evB));

  double total_start = now_ms();
  double inertia = 0.0;

  for(int iter=0; iter<max_iter; ++iter){
    double t_iter0 = now_ms();

    // zero accumulators
    CUDA_CHECK(cudaMemset(d_sums, 0, szC));
    CUDA_CHECK(cudaMemset(d_counts, 0, K*sizeof(int)));

    // ---- assignment + accumulate (kernel) ----
    CUDA_CHECK(cudaEventRecord(evA));
    assign_accumulate_kernel<<<grid, block_size>>>(
      d_data, d_centroids, N, D, K, d_labels, d_sums, d_counts, d_dist2, d_threadmap
    );
    CUDA_CHECK(cudaEventRecord(evB));
    CUDA_CHECK(cudaEventSynchronize(evB));
    float t_kernel_ms=0.0f; CUDA_CHECK(cudaEventElapsedTime(&t_kernel_ms, evA, evB));

    // ---- update centroids (host) ----
    double t0 = now_ms();
    // copy sums + counts back
    double *h_sums=(double*)malloc(szC);
    int *h_counts=(int*)malloc(K*sizeof(int));
    CUDA_CHECK(cudaMemcpy(h_sums, d_sums, szC, cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_counts, d_counts, K*sizeof(int), cudaMemcpyDeviceToHost));

    double movement=0.0;
    for(int k=0;k<K;++k){
      if(h_counts[k]>0){
        for(int d=0; d<D; ++d){
          double newc = h_sums[k*D + d] / (double)h_counts[k];
          double diff = h_centroids[k*D + d] - newc;
          movement += diff*diff;
          h_centroids[k*D + d] = newc;
        }
      } else {
        // keep old centroid if empty
      }
    }
    // copy new centroids to device
    CUDA_CHECK(cudaMemcpy(d_centroids, h_centroids, szC, cudaMemcpyHostToDevice));
    double t_update_ms = now_ms() - t0;

    // ---- inertia (host sum over dist2) ----
    t0 = now_ms();
    // We already computed nearest-centroid distances in kernel into d_dist2.
    double *h_d2 = (double*)malloc(N*sizeof(double));
    CUDA_CHECK(cudaMemcpy(h_d2, d_dist2, N*sizeof(double), cudaMemcpyDeviceToHost));
    inertia = 0.0;
    for(int i=0;i<N;++i) inertia += h_d2[i];
    free(h_d2);
    double t_inertia_ms = now_ms() - t0;

    double t_iter_ms = now_ms() - t_iter0;

    if(verbose){
      if(iter==0){
        int thrmap[16*3]={0};
        CUDA_CHECK(cudaMemcpy(thrmap, d_threadmap, 16*3*sizeof(int), cudaMemcpyDeviceToHost));
        printf("Thread map sample (first up to 16 points): [i, blockIdx.x, threadIdx.x]\n");
        int sample = (N<16)?N:16;
        for(int j=0;j<sample;++j){
          printf("  i=%d  block=%d  thread=%d\n", thrmap[j*3+0], thrmap[j*3+1], thrmap[j*3+2]);
        }
      }
      printf("Iter %3d | kernel(assign+accum)=%.3f ms, update=%.3f ms, inertia(sum)=%.3f ms | total=%.3f ms | move=%.6g\n",
             iter+1, t_kernel_ms, t_update_ms, t_inertia_ms, t_iter_ms, movement);
    }
    if(pf){ fprintf(pf, "%d,%.6f,%.6f,%.6f,%.6f\n",
                    iter+1, t_kernel_ms, t_update_ms, t_inertia_ms, t_iter_ms); fflush(pf); }

    free(h_sums); free(h_counts);

    if(movement < tol){
      if(verbose) printf("Converged (movement %.6g <= tol %.6g)\n", movement, tol);
      break;
    }
  }

  double total_ms = now_ms() - total_start;
  printf("Total wall time: %.2f ms  (includes H2D/D2H where applicable)\n", total_ms);

  // Copy final labels back and save outputs
  CUDA_CHECK(cudaMemcpy(h_labels, d_labels, szL, cudaMemcpyDeviceToHost));
  save_labels("labels_cuda.csv", h_labels, N);
  save_centroids("centroids_cuda.csv", h_centroids, K, D);

  // meta for viz
  FILE* meta=fopen("meta_cuda.txt","w");
  if(meta){ fprintf(meta, "N %d\nD %d\nK %d\n", N,D,K); fclose(meta); }
  if(pf){ fclose(pf); printf("Profile saved to %s\n", profile_csv); }

  // cleanup
  cudaEventDestroy(evA); cudaEventDestroy(evB);
  cudaFree(d_data); cudaFree(d_centroids); cudaFree(d_labels);
  cudaFree(d_sums); cudaFree(d_counts); cudaFree(d_dist2); cudaFree(d_threadmap);
  free(h_data); free(h_centroids); free(h_labels);
  return 0;
}

Writing kmeans_cuda_prof.cu


In [ ]:
!nvcc -O3 -arch=sm_75 kmeans_cuda_prof.cu -o kmeans_cuda_prof
!ls -lh kmeans_cuda_prof

-rwxr-xr-x 1 root root 1000K Nov 29 22:08 kmeans_cuda_prof


In [ ]:
import subprocess
import re
import csv
import time

# Ns you want to test
Ns = [100, 500, 1000, 5000, 10000, 20000, 50000, 100000, 200000, 300000, 500000, 1000000, 2000000]

D = 16
K = 10
max_iter = 200
tol = "1e-4"
block = 256
verbose = 0   # KEEP THIS 0 FOR SPEED

results = []

print("Running K-means benchmarks...\n")

for N in Ns:
    print(f"▶ Running N = {N:,} ...")

    # build command
    cmd = ["./kmeans_cuda_prof",
           str(N), str(D), str(K),
           str(max_iter), tol,
           str(block), str(verbose),
           f"profile_{N}.csv"]

    # run & capture output
    start = time.time()
    out = subprocess.check_output(cmd, universal_newlines=True)
    end = time.time()

    # find "Total wall time: xxx ms"
    match = re.search(r"Total wall time:\s*([\d.]+)\s*ms", out)
    total_ms = float(match.group(1)) if match else None

    results.append((N, total_ms))
    print(f"   ✔ Done: {total_ms} ms\n")

print("🎉 All runs complete!\n")

# Save results to a CSV
with open("runtime_results.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["N", "time_ms"])
    writer.writerows(results)

print("Saved results to runtime_results.csv\n")

# Also print a summary
print("===== SUMMARY =====")
for N, t in results:
    print(f"N={N:>10} → {t:.3f} ms")



Running K-means benchmarks...

▶ Running N = 100 ...
   ✔ Done: 0.79 ms

▶ Running N = 500 ...
   ✔ Done: 4.5 ms

▶ Running N = 1,000 ...
   ✔ Done: 9.04 ms

▶ Running N = 5,000 ...
   ✔ Done: 62.22 ms

▶ Running N = 10,000 ...
   ✔ Done: 48.74 ms

▶ Running N = 20,000 ...
   ✔ Done: 80.0 ms

▶ Running N = 50,000 ...
   ✔ Done: 201.36 ms

▶ Running N = 100,000 ...
   ✔ Done: 255.15 ms

▶ Running N = 200,000 ...
   ✔ Done: 401.4 ms

▶ Running N = 300,000 ...
   ✔ Done: 577.26 ms

▶ Running N = 500,000 ...
   ✔ Done: 926.25 ms

▶ Running N = 1,000,000 ...
   ✔ Done: 1936.34 ms

▶ Running N = 2,000,000 ...
   ✔ Done: 3547.97 ms

🎉 All runs complete!

Saved results to runtime_results.csv

===== SUMMARY =====
N=       100 → 0.790 ms
N=       500 → 4.500 ms
N=      1000 → 9.040 ms
N=      5000 → 62.220 ms
N=     10000 → 48.740 ms
N=     20000 → 80.000 ms
N=     50000 → 201.360 ms
N=    100000 → 255.150 ms
N=    200000 → 401.400 ms
N=    300000 → 577.260 ms
N=    500000 → 926.250 ms
N=   1000

In [ ]:
!nvcc -O3 -arch=sm_75 kmeans_cuda_prof.cu -o kmeans_cuda_prof
!gcc -O3 -march=native kmeans_cpu_serial.c -o kmeans_cpu_serial -lm


cc1: fatal error: kmeans_cpu_serial.c: No such file or directory
compilation terminated.


In [ ]:
# ============================================================
# N-WISE GPU + CPU MONITORING FOR CUDA K-MEANS
# Saves: nwise_monitoring.csv
# ============================================================

import subprocess
import time
import pandas as pd
import psutil
import threading

# ------------------------------------------------------------
# 1. CONFIGURE YOUR N VALUES
# ------------------------------------------------------------
N_values = [100, 500, 1000, 5000, 10000, 20000, 50000, 100000, 200000, 300000, 500000, 1000000, 2000000]

# ------------------------------------------------------------
# 2. FUNCTION TO RUN GPU MONITOR
# ------------------------------------------------------------
gpu_records = []
monitor_running = True

def gpu_monitor():
    global monitor_running
    while monitor_running:
        # Run nvidia-smi once
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=utilization.gpu,utilization.memory,memory.used,power.draw",
             "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE, text=True
        )
        gpu = result.stdout.strip().split(", ")
        if len(gpu) == 4:
            gpu_records.append({
                "util_gpu": float(gpu[0]),
                "util_mem": float(gpu[1]),
                "mem_used": float(gpu[2]),
                "power": float(gpu[3]),
            })
        time.sleep(0.1)  # 0.1 sec sampling

# ------------------------------------------------------------
# 3. FUNCTION TO RUN CPU MONITOR
# ------------------------------------------------------------
cpu_records = []

def cpu_monitor():
    global monitor_running
    while monitor_running:
        cpu_records.append({
            "cpu": psutil.cpu_percent(interval=0.1),
            "ram": psutil.virtual_memory().used / (1024**2),
        })

# ------------------------------------------------------------
# 4. RUN EXPERIMENT FOR EACH N
# ------------------------------------------------------------
rows = []

for N in N_values:
    print(f"\n===== Running N = {N} =====")

    gpu_records = []
    cpu_records = []
    monitor_running = True

    # Start monitoring threads
    t1 = threading.Thread(target=gpu_monitor)
    t2 = threading.Thread(target=cpu_monitor)
    t1.start()
    t2.start()

    # Run CUDA program (adjust if needed)
    cmd = ["./kmeans_cuda_prof", str(N), "16", "10", "200", "1e-4", "256", "0", "profile.csv"]
    start = time.time()
    subprocess.run(cmd)
    end = time.time()

    # Stop monitors
    monitor_running = False
    t1.join()
    t2.join()

    # Convert to DataFrames
    if len(gpu_records) == 0:
        print("Warning: GPU monitor captured no samples!")
        continue

    df_gpu = pd.DataFrame(gpu_records)
    df_cpu = pd.DataFrame(cpu_records)

    # Compute metrics
    gpu_avg = df_gpu["util_gpu"].mean()
    gpu_max = df_gpu["util_gpu"].max()
    mem_avg = df_gpu["mem_used"].mean()
    power_avg = df_gpu["power"].mean()

    cpu_avg = df_cpu["cpu"].mean()
    ram_avg = df_cpu["ram"].mean()

    # Add row
    rows.append({
        "N": N,
        "time_ms": (end - start) * 1000,
        "gpu_util_avg": gpu_avg,
        "gpu_util_max": gpu_max,
        "gpu_mem_avg_MB": mem_avg,
        "gpu_power_avg_W": power_avg,
        "cpu_util_avg": cpu_avg,
        "ram_used_avg_MB": ram_avg,
    })

# ------------------------------------------------------------
# 5. SAVE RESULTS
# ------------------------------------------------------------
df_final = pd.DataFrame(rows)
df_final.to_csv("nwise_monitoring.csv", index=False)

print("\nDONE! Saved N-wise metrics to nwise_monitoring.csv")
df_final



===== Running N = 100 =====

===== Running N = 500 =====

===== Running N = 1000 =====

===== Running N = 5000 =====

===== Running N = 10000 =====

===== Running N = 20000 =====

===== Running N = 50000 =====

===== Running N = 100000 =====

===== Running N = 200000 =====

===== Running N = 300000 =====

===== Running N = 500000 =====

===== Running N = 1000000 =====

===== Running N = 2000000 =====

DONE! Saved N-wise metrics to nwise_monitoring.csv


,N,time_ms,gpu_util_avg,gpu_util_max,gpu_mem_avg_MB,gpu_power_avg_W,cpu_util_avg,ram_used_avg_MB
0,100,190.271378,0.000000,0.0,10.000000,15.005000,56.600000,862.498047
1,500,117.279530,0.000000,0.0,2.000000,25.770000,33.950000,870.294922
2,1000,118.316889,5.000000,5.0,2.000000,25.770000,37.250000,870.078125
3,5000,165.467262,5.000000,5.0,52.000000,25.775000,54.100000,882.542969
4,10000,188.247919,16.000000,16.0,53.000000,25.825000,63.050000,873.318359
5,20000,241.775990,10.500000,18.0,55.000000,25.970000,48.266667,894.290365
6,50000,461.840630,41.000000,87.0,59.000000,28.527500,57.240000,910.171875
7,100000,635.420322,35.166667,87.0,40.333333,33.585000,55.728571,897.000558
8,200000,1020.901203,29.555556,82.0,46.222222,40.105556,56.181818,905.813210
9,300000,1447.608709,23.250000,83.0,62.000000,41.619167,58.360000,916.994271


In [ ]:
import subprocess, time, psutil
import csv

Ns = [100,500,1000,5000,10000,20000,50000,100000,200000,300000,500000,1000000,2000000]

rows = []
for N in Ns:
    print(f"Running SERIAL for N={N}...")

    cpu_util = []
    ram_used = []

    # start serial program
    p = subprocess.Popen(
        ["./kmeans_cpu_serial", str(N), "16", "10", "200", "1e-4", "1", "0", "profile_cpu_serial.csv"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )

    start = time.time()

    # monitor only CPU + RAM (no GPU here)
    while p.poll() is None:
        cpu_util.append(psutil.cpu_percent(interval=0.1))
        ram_used.append(psutil.virtual_memory().used / (1024*1024))

    end = time.time()

    time_ms = (end - start) * 1000

    rows.append([
        N, time_ms,
        sum(cpu_util)/len(cpu_util),
        sum(ram_used)/len(ram_used)
    ])

# save CPU monitoring file
with open("serial_monitoring.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["N","serial_time_ms","cpu_util_avg","ram_used_avg_MB"])
    w.writerows(rows)

print("DONE — saved serial_monitoring.csv")


Running SERIAL for N=100...
Running SERIAL for N=500...
Running SERIAL for N=1000...
Running SERIAL for N=5000...
Running SERIAL for N=10000...
Running SERIAL for N=20000...
Running SERIAL for N=50000...
Running SERIAL for N=100000...
Running SERIAL for N=200000...
Running SERIAL for N=300000...
Running SERIAL for N=500000...
Running SERIAL for N=1000000...
Running SERIAL for N=2000000...
DONE — saved serial_monitoring.csv


In [ ]:
!nvcc -O3 -arch=sm_75 kmeans_cuda_prof.cu -o kmeans_cuda_prof
!ls -lh kmeans_cuda_prof

-rwxr-xr-x 1 root root 1000K Nov 29 11:54 kmeans_cuda_prof


In [ ]:
!./kmeans_cuda_prof 200000 16 10 200 1e-4 256 1 profile_cuda.csv

Device 0: Tesla T4
  CC 7.5  SMs=40  warpSize=32  maxThreadsPerBlock=1024  sharedMemPerBlock=48 KB
  memory: 15.8 GB global, 0.1 MB const
K-means CUDA: N=200000 D=16 K=10 max_iter=200 tol=0.0001 block=256 verbose=1
Generating synthetic data on host...
Init (kmeans++) on host...
Launch config: grid=782 blocks, block=256 threads  (total threads 200192)
Thread map sample (first up to 16 points): [i, blockIdx.x, threadIdx.x]
  i=0  block=0  thread=0
  i=1  block=0  thread=1
  i=2  block=0  thread=2
  i=3  block=0  thread=3
  i=4  block=0  thread=4
  i=5  block=0  thread=5
  i=6  block=0  thread=6
  i=7  block=0  thread=7
  i=8  block=0  thread=8
  i=9  block=0  thread=9
  i=10  block=0  thread=10
  i=11  block=0  thread=11
  i=12  block=0  thread=12
  i=13  block=0  thread=13
  i=14  block=0  thread=14
  i=15  block=0  thread=15
Iter   1 | kernel(assign+accum)=3.945 ms, update=0.040 ms, inertia(sum)=1.279 ms | total=5.293 ms | move=89.6335
Iter   2 | kernel(assign+accum)=3.719 ms, update=0

In [ ]:
# Visualization for CUDA run (reads labels_cuda.csv, centroids_cuda.csv, meta_cuda.txt)
import os, math, numpy as np, matplotlib.pyplot as plt, imageio
from matplotlib import colormaps

def load_meta(path="meta_cuda.txt"):
    vals={}
    with open(path,'r') as f:
        for ln in f:
            parts=ln.strip().split()
            if len(parts)==2:
                vals[parts[0]]=int(parts[1])
    return vals['N'], vals['D'], vals['K']

def safe_load_centroids(path):
    return np.loadtxt(path, dtype=float, ndmin=2, delimiter=',')

# RNG to reproduce exact C synthetic set
class XorShift64Star:
    def __init__(self, seed=88172645463325252):
        self.state = seed & ((1<<64)-1)
    def next_u64(self):
        x = self.state
        x ^= (x >> 12) & ((1<<64)-1)
        x ^= (x << 25) & ((1<<64)-1)
        x ^= (x >> 27) & ((1<<64)-1)
        self.state = x & ((1<<64)-1)
        return (x * 2685821657736338717) & ((1<<64)-1)
    def drand01(self):
        u = self.next_u64()
        v = (u >> 11) & ((1<<53)-1)
        return v * (1.0/9007199254740992.0)

def generate_synthetic_same(N, D, K, seed=88172645463325252):
    rng = XorShift64Star(seed)
    centers = np.zeros((K, D), dtype=np.float64)
    for k in range(K):
        for d in range(D):
            centers[k, d] = (rng.drand01() * 2.0 - 1.0) * 10.0
    data = np.zeros((N, D), dtype=np.float64)
    for i in range(N):
        c = i % K
        for d in range(D):
            u1 = rng.drand01()
            u2 = rng.drand01()
            r = math.sqrt(-2.0 * math.log(max(u1, 1e-12)))
            z = r * math.cos(2.0 * math.pi * u2)
            scale = 1.0 + 0.5 * rng.drand01()
            data[i, d] = centers[c, d] + z * scale
    return data

def pca_project_2d(X):
    Xc = X - np.mean(X, axis=0, keepdims=True)
    C = (Xc.T @ Xc) / (Xc.shape[0]-1)
    vals, vecs = np.linalg.eigh(C)
    order = np.argsort(vals)[::-1]
    top2 = vecs[:, order[:2]]
    proj = Xc @ top2
    return proj, top2, Xc.mean(axis=0, keepdims=True)

# --- load outputs ---
if not (os.path.exists("labels_cuda.csv") and os.path.exists("centroids_cuda.csv") and os.path.exists("meta_cuda.txt")):
    raise FileNotFoundError("Missing CUDA outputs. Run the CUDA binary first.")

N,D,K = load_meta("meta_cuda.txt")
labels = np.loadtxt("labels_cuda.csv", dtype=int, ndmin=1)
centroids = safe_load_centroids("centroids_cuda.csv")
assert labels.shape[0]==N, "labels size mismatch with meta"

# regenerate data exactly
data = generate_synthetic_same(N,D,K)

# project to 2D for plot
if D==2:
    coords = data.copy()
    cent2 = centroids.copy()
    xlabel, ylabel = "x","y"
else:
    coords, basis, mean0 = pca_project_2d(data)
    cent2 = (centroids - mean0) @ basis
    xlabel, ylabel = "PC 1","PC 2"

# plotting (uniform size)
os.makedirs("outputs", exist_ok=True)
cmap = colormaps.get_cmap('tab20')
colors = [cmap(i % cmap.N) for i in range(K)]

def save_plot(path, zoom=False):
    plt.figure(figsize=(10,8))
    if zoom:
        pad=0.05
        xmin,xmax = coords[:,0].min(), coords[:,0].max()
        ymin,ymax = coords[:,1].min(), coords[:,1].max()
        dx,dy=(xmax-xmin),(ymax-ymin)
        if dx==0: dx=1.0
        if dy==0: dy=1.0
        plt.xlim(xmin-pad*dx, xmax+pad*dx)
        plt.ylim(ymin-pad*dy, ymax+pad*dy)

    for k in range(K):
        m = (labels==k)
        pts = coords[m]
        if pts.size>0:
            plt.scatter(pts[:,0], pts[:,1], s=8, color=colors[k], alpha=0.6, label=f"C{k}")
    plt.scatter(cent2[:,0], cent2[:,1], s=180, color='black', marker='X', label='Centroids')
    for k in range(K):
        plt.text(cent2[k,0], cent2[k,1], f" {k}", color='white', fontsize=9,
                 bbox=dict(facecolor='black', alpha=0.7, boxstyle='round,pad=0.2'))
    plt.title(f"K-means (CUDA): N={N}, D={D}, K={K}")
    plt.xlabel(xlabel); plt.ylabel(ylabel)
    plt.legend(markerscale=2, fontsize='small', ncol=2)
    plt.tight_layout()
    plt.savefig(path, dpi=200); plt.close()
    plt.show()
full_png = "outputs/kmeans_cuda_viz.png"
zoom_png = "outputs/kmeans_cuda_viz_zoom.png"
save_plot(full_png, zoom=False)
save_plot(zoom_png, zoom=True)
print("Saved", full_png, "and", zoom_png)

# build a small GIF (same size frames)
frames = [imageio.imread(full_png), imageio.imread(zoom_png)]
imageio.mimsave("outputs/kmeans_cuda_viz.gif", frames, duration=1.0)
print("Saved outputs/kmeans_cuda_viz.gif")

# peek a few coords
print("Sample:", [(int(i), int(labels[i]), np.round(coords[i,:],3).tolist()) for i in range(min(5,N))])

Saved outputs/kmeans_cuda_viz.png and outputs/kmeans_cuda_viz_zoom.png


/tmp/ipython-input-789954753.py:121: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames = [imageio.imread(full_png), imageio.imread(zoom_png)]


Saved outputs/kmeans_cuda_viz.gif
Sample: [(0, 8, [-9.834, 11.104]), (1, 2, [17.624, -4.989]), (2, 3, [0.112, 14.536]), (3, 0, [-11.953, -8.056]), (4, 8, [-1.123, 4.02])]
